In [ ]:
# This is a tool to explain what a code snippet means and it's function using open ai gpt-4o-mini model

# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'

In [ ]:
# set up environment
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes what a provided code samples does. \
Please explain what this code does and why? provides a short summary, ignoring text that might be navigation related. \
try as much as possible not to exceed 1000 words. \
Respond in markdown."

In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(question):
    user_prompt = f"You are looking at a code sample provided"
    # user_prompt += "\n"
    user_prompt += f"{question}"
    return user_prompt

In [ ]:
# Format message content

def messages_for(question):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(question)}
    ]

In [ ]:
# Get gpt-4o-mini to answer, with streaming
def stream_code_explain(question):
    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages_for(question),
        # below code is use to make typewriter feature enable when returning the response
        # if you want to make use of it in claude.ai you won't use this key                                    
        stream=True)
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_code_explain(question)